In [1]:
import pandas as pd
df_dates = pd.read_csv(r'D:\Programming\Download_Backtest_Deploy_data\1__Download\TradingView_data_download\storage\ADANIENT_historical_data.csv')
df_dates['Date'] = pd.to_datetime(df_dates['Date'], errors='coerce').dt.date
date_str_list = [str(d) for d in df_dates['Date'].values.tolist()]
date_str_list

['2021-01-01',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-25',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-03-01',
 '2021-03-02',
 '2021-03-03',
 '2021-03-04',
 '2021-03-05',
 '2021-03-08',
 '2021-03-09',
 '2021-03-10',
 '2021-03-12',
 '2021-03-15',
 '2021-03-16',
 '2021-03-17',
 '2021-03-18',
 '2021-03-19',
 '2021-03-22',
 '2021-03-23',
 '2021-03-24',
 '2021-03-25',
 '2021-03-26',
 '2021-03-30',
 '2021-03-31',
 '2021-04-01',
 '2021-04-05',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from datetime import datetime, timedelta
import time
import os
import glob

class NSEArchivesDownloader:
    def __init__(self, download_dir=None):
        if download_dir is None:
            download_dir = os.path.join(os.getcwd(), "nse_downloads")

        download_dir = os.path.abspath(download_dir)

        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "directory_upgrade": True,
            "safebrowsing.enabled": True,
            "profile.default_content_settings.popups": 0
        }
        chrome_options.add_experimental_option("prefs", prefs)
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        self.driver = webdriver.Chrome(options=chrome_options)
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        self.wait = WebDriverWait(self.driver, 30)
        self.download_dir = download_dir
        self.date_str_list = []  # Store dates as list

    def wait_for_downloads(self, timeout=30):
        """Wait for all downloads to complete"""
        seconds = 0
        dl_wait = True
        while dl_wait and seconds < timeout:
            time.sleep(1)
            dl_wait = False
            for fname in os.listdir(self.download_dir):
                if fname.endswith('.crdownload') or fname.endswith('.tmp'):
                    dl_wait = True
            seconds += 1
        return True

    def check_invalid_files(self):
        """Check if invalid date zip files exist (don't delete)"""
        pattern = os.path.join(self.download_dir, "*Invalid date*.zip")
        invalid_files = glob.glob(pattern)
        return len(invalid_files)

    def navigate_to_indices_section(self):
        """First click on Indices tab in left navigation"""
        try:
            time.sleep(5)
            indices_tab = self.wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a#reportleftnav2[href="#cr_indices"]'))
            )
            self.driver.execute_script("arguments[0].click();", indices_tab)
            time.sleep(3)
            print("✓ Clicked Indices tab")
            return True
        except Exception as e:
            print(f"✗ Error: {e}")
            return False

    def navigate_to_archives_tab(self):
        """Navigate to the Archives tab within Indices section"""
        try:
            archives_tab = self.wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#cr_indices_archives"]'))
            )
            self.driver.execute_script("arguments[0].click();", archives_tab)
            time.sleep(3)
            print("✓ Navigated to Archives tab")
            return True
        except Exception as e:
            print(f"✗ Error: {e}")
            return False

    def select_date(self, date_str):
        """Select a specific date in DD-MM-YYYY format"""
        try:
            date_input = self.wait.until(
                EC.presence_of_element_located((By.ID, "cr_indices_archives_date"))
            )
            
            self.driver.execute_script(f"""
                var input = arguments[0];
                input.value = '{date_str}';
                
                var changeEvent = new Event('change', {{ bubbles: true }});
                var inputEvent = new Event('input', {{ bubbles: true }});
                var blurEvent = new Event('blur', {{ bubbles: true }});
                
                input.dispatchEvent(inputEvent);
                input.dispatchEvent(changeEvent);
                input.dispatchEvent(blurEvent);
                
                if (typeof $(input).trigger === 'function') {{
                    $(input).trigger('change');
                }}
            """, date_input)
            
            time.sleep(3)
            return True

        except Exception as e:
            return False

    def select_all_reports(self):
        """Click 'Select All Reports' checkbox"""
        try:
            select_all = self.wait.until(
                EC.presence_of_element_located((
                    By.XPATH,
                    "//div[@id='cr_indices_archives']//label[@class='chk_container']//input[@class='check_uncheck']"
                ))
            )

            self.driver.execute_script("arguments[0].scrollIntoView(true);", select_all)
            time.sleep(0.5)

            if not select_all.is_selected():
                self.driver.execute_script("arguments[0].click();", select_all)
                time.sleep(1)

            selected = self.driver.find_elements(By.CSS_SELECTOR,
                "div#cr_indices_archives input[type='checkbox']:checked")
            
            return len(selected)

        except Exception as e:
            return 0

    def click_multiple_download(self):
        """Click the 'Multiple file Download' button"""
        try:
            files_before = set(os.listdir(self.download_dir))
            invalid_before = self.check_invalid_files()
            
            download_btn = self.wait.until(
                EC.presence_of_element_located((
                    By.XPATH,
                    "//div[@id='cr_indices_archives']//a[contains(@onclick, 'downloadReports')]"
                ))
            )

            self.driver.execute_script("arguments[0].scrollIntoView(true);", download_btn)
            time.sleep(0.5)
            self.driver.execute_script("arguments[0].click();", download_btn)
            
            time.sleep(5)
            self.wait_for_downloads(timeout=20)
            
            files_after = set(os.listdir(self.download_dir))
            new_files = files_after - files_before
            invalid_after = self.check_invalid_files()
            
            # Check if invalid files were created
            invalid_count = invalid_after - invalid_before
            
            # Return valid files count and whether invalid file was created
            valid_new_files = len(new_files) - invalid_count
            has_invalid = invalid_count > 0
            
            return valid_new_files, has_invalid

        except Exception as e:
            return 0, False

    def download_for_date(self, date_str):
        """Complete download process for a specific date"""
        if not self.select_date(date_str):
            return False, 0, False

        checkboxes = self.select_all_reports()
        if checkboxes == 0:
            return True, 0, False  # No reports for this date

        new_files, has_invalid = self.click_multiple_download()
        
        return True, new_files, has_invalid

    def generate_date_range(self, start_date_str, end_date_str):
        """Generate list of dates from start to end"""
        start_date = datetime.strptime(start_date_str, "%d-%m-%Y")
        end_date = datetime.strptime(end_date_str, "%d-%m-%Y")

        date_str_list = []
        current_date = start_date

        while current_date <= end_date:
            date_str_list.append(current_date.strftime("%d-%m-%Y"))
            current_date += timedelta(days=1)

        return date_str_list

    def run(self, url, start_date="01-01-2021", end_date=None):
        """Main execution method"""
        if end_date is None:
            end_date = datetime.now().strftime("%d-%m-%Y")

        try:
            print(f"Opening: {url}")
            self.driver.get(url)
            
            if not self.navigate_to_indices_section():
                raise Exception("Failed to navigate to Indices")

            if not self.navigate_to_archives_tab():
                raise Exception("Failed to navigate to Archives")

            # Generate date list and store in date_str_list
            self.date_str_list = self.generate_date_range(start_date, end_date)
            print(f"\nDates: {len(self.date_str_list)} ({start_date} to {end_date})")
            print(f"Path: {self.download_dir}\n")

            successful = 0
            failed = 0
            no_data_count = 0
            total_files_downloaded = 0

            for i, date_str in enumerate(self.date_str_list, 1):
                print(f"[{i}/{len(self.date_str_list)}] {date_str}...", end=" ")

                try:
                    success, new_files, has_invalid = self.download_for_date(date_str)
                    
                    if success:
                        if new_files > 0:
                            successful += 1
                            total_files_downloaded += new_files
                            print(f"✓ {new_files} files")
                        elif has_invalid:
                            no_data_count += 1
                            print("✓ File not found (Invalid date file)")
                        else:
                            no_data_count += 1
                            print("✓ File not found")
                    else:
                        failed += 1
                        print("✗ Failed")

                    # time.sleep(2)

                    if i < len(self.date_str_list):
                        time.sleep(1)

                except Exception as e:
                    print(f"✗ Error")
                    failed += 1

            print(f"\n{'='*60}")
            print(f"SUMMARY")
            print(f"{'='*60}")
            print(f"Total dates: {len(self.date_str_list)}")
            print(f"Files downloaded: {successful} dates ({total_files_downloaded} files)")
            print(f"File not found: {no_data_count}")
            print(f"Failed: {failed}")
            print(f"\nPath: {self.download_dir}")
            print(f"{'='*60}")

        except Exception as e:
            print(f"✗ Error: {e}")
            import traceback
            traceback.print_exc()

        finally:
            input("\nPress Enter to close...")
            self.driver.quit()


# Run for all dates from 01-01-2021 to today
if __name__ == "__main__":
    NSE_URL = "https://www.nseindia.com/all-reports"
    download_path = os.path.join(os.getcwd(), "nse_downloads")
    downloader = NSEArchivesDownloader(download_dir=download_path)
    
    downloader.run(
        url=NSE_URL,
        start_date="13-07-2024",
        end_date=None  # None = today
    )

Opening: https://www.nseindia.com/all-reports
✓ Clicked Indices tab
✓ Navigated to Archives tab

Dates: 450 (13-07-2024 to 05-10-2025)
Path: d:\Programming\Download_Backtest_Deploy\1__Download\NSE_data_download\nse_downloads

[1/450] 13-07-2024... ✓ File not found (Invalid date file)
[2/450] 14-07-2024... ✓ File not found (Invalid date file)
[3/450] 15-07-2024... ✓ File not found (Invalid date file)
[4/450] 16-07-2024... ✓ File not found (Invalid date file)
[5/450] 17-07-2024... ✓ File not found (Invalid date file)
[6/450] 18-07-2024... ✓ File not found (Invalid date file)
[7/450] 19-07-2024... ✓ File not found (Invalid date file)
[8/450] 20-07-2024... ✓ File not found (Invalid date file)
[9/450] 21-07-2024... ✓ File not found (Invalid date file)
[10/450] 22-07-2024... ✓ File not found (Invalid date file)
[11/450] 23-07-2024... ✓ File not found (Invalid date file)
[12/450] 24-07-2024... ✓ File not found (Invalid date file)
[13/450] 25-07-2024... ✓ File not found (Invalid date file)
[14